In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from copy import copy
from multiprocessing import Pool, Process, Value
import time
from threading import Thread

In [2]:
filename = 'CRTRAITS.TXT'

data = pd.read_csv(filename, sep='\t', encoding='utf-8')
data.head()

,Singular,Plural,Wood,Mercury,Ore,Sulfur,Crystal,Gems,Gold,Fight Value,...,Attack,Defense,MinDmg,MaxDmg,Shots,Spells,GuardsLow,GuardsHigh,Ability Text,Attributes
0,Pikinier,Pikinierzy,0,0,0,0,0,0,60,100,...,4,5,1,3,0,0,20,50,Powstrzymuje szarżę.,0
1,Halabardnik,Halabardnicy,0,0,0,0,0,0,75,115,...,6,5,2,3,0,0,20,30,Powstrzymuje szarżę.,0
2,Łucznik,Łucznicy,0,0,0,0,0,0,100,115,...,6,3,2,3,12,0,16,30,NaN,SHOOTING_ARMY
3,Kusznik,Kusznicy,0,0,0,0,0,0,150,115,...,6,3,2,3,24,0,16,25,Strzela dwukrotnie.,const_two_attacks | SHOOTING_ARMY
4,Gryf,Gryfy,0,0,0,0,0,0,200,324,...,8,8,3,6,0,0,12,25,Dwukrotnie kontratakuje.,DOUBLE_WIDE | FLYING_ARMY


In [3]:
data.Attributes.fillna(u'0', inplace=True)
data.Attributes.values[data.Attributes.values == u'0'] = ''
set(' | '.join(data.Attributes.values).split(' | '))

{u'',
 u'CATAPULT',
 u'DOUBLE_WIDE',
 u'FLYING_ARMY',
 u'HAS_EXTENDED_ATTACK',
 u'IMMUNE_TO_FIRE_SPELLS',
 u'IMMUNE_TO_MIND_SPELLS',
 u'IS_UNDEAD',
 u'KING_1',
 u'KING_2',
 u'KING_3',
 u'MULTI_HEADED',
 u'SHOOTING_ARMY',
 u'SIEGE_WEAPON',
 u'const_free_attack',
 u'const_jousting',
 u'const_lowers_morale',
 u'const_no_melee_penalty',
 u'const_no_wall_penalty',
 u'const_raises_morale',
 u'const_two_attacks'}

In [4]:
# do wzięcia pod uwagę w symulacjach 1vs1
keywords = {'DOUBLE_WIDE', 
            'SHOOTING_ARMY', 
            'const_free_attack', 
            'const_jousting',
            'const_no_melee_penalty',
            'const_two_attacks'}

In [5]:
name = ['Singular']
crap = ['Plural', 'Wood', 'Mercury', 'Ore', 'Sulfur', 'Crystal', 'Gems', 'Gold', 'Ability Text']
growth = ['Growth', 'Horde Growth']
abilities = ['Attributes']
spells = ['Spells']
guards_quantity = ['GuardsLow', 'GuardsHigh']

cols_to_drop = crap + growth + guards_quantity + spells
data.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
data.head()

,Singular,Fight Value,AI Value,Hit Points,Speed,Attack,Defense,MinDmg,MaxDmg,Shots,Attributes
0,Pikinier,100,80,10,4,4,5,1,3,0,
1,Halabardnik,115,115,10,5,6,5,2,3,0,
2,Łucznik,115,126,10,4,6,3,2,3,12,SHOOTING_ARMY
3,Kusznik,115,184,10,6,6,3,2,3,24,const_two_attacks | SHOOTING_ARMY
4,Gryf,324,351,25,6,8,8,3,6,0,DOUBLE_WIDE | FLYING_ARMY


In [7]:
class unit_type(object):
    def __init__(self, name, fightv, aiv, hp, spd, att, df, dmlow, dmhi, shots, abi):
        self.name = name
        self.fight_value = fightv
        self.ai_value = aiv
        self.hp = hp
        self.speed = spd
        self.attack = att
        self.defense = df
        self.dmg_min = dmlow
        self.dmg_max = dmhi
        self.shots = shots
        self.attributes = {x for x in abi.split(' | ') if x in keywords}
        
class stack(object):
    def __init__(self, unit, count):
        self.count = count
        self.name = unit.name
        
        self.hp = unit.hp
        self.hp_left = self.hp
        
        self.speed = unit.speed
        self.attack = unit.attack
        self.defense = unit.defense
        self.dmg_min = unit.dmg_min
        self.dmg_max = unit.dmg_max
        self.shots = unit.shots
        self.attributes = unit.attributes      
        
        
    def take_dmg(self, dmg):
        if dmg < self.hp_left:
            self.hp_left -= dmg
        else:
            dmg -= self.hp_left
            num_killed, rem = divmod(dmg, self.hp)
            self.count -= num_killed + 1
            self.hp_left = self.hp - rem
            self.count = max(self.count, 0)
            
            
    def __calc_base_damage(self, other):
        if self.count < 10:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=self.count))
        else:
            base_dmg = sum(np.random.randint(self.dmg_min, self.dmg_max+1, 
                                             size=10)) * self.count / 10
        
        att_to_def = self.attack - other.defense
        base_dmg_reduction = 0.
        base_dmg_bonus = 0.
        if att_to_def > 0:
            base_dmg_bonus = min(.05 * att_to_def, 3.)
        else:
            base_dmg_reduction = min(.025 * -att_to_def, .7)
            
        return base_dmg, base_dmg_bonus, base_dmg_reduction
    
            
    def attack_melee(self, other, dmg_bonus=0., melee_penalty=False):
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        if self.name == 'Upiorny rycerz' and np.random.rand() < .2:
            dmg_bonus += 1.
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if melee_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        
        
    def attack_range(self, other, dmg_bonus=0., range_penalty=False):
        assert self.is_shooter() and self.shots > 0
        base_dmg, base_dmg_bonus, base_dmg_reduction = self.__calc_base_damage(other)
        dmg_bonus += base_dmg_bonus
            
        damage = base_dmg * (1. + dmg_bonus) * (1. - base_dmg_reduction)
        if range_penalty:
            damage /= 2.
            
        other.take_dmg(int(damage))
        self.shots -= 1
        
    def is_alive(self):
        return self.count > 0
        
    def is_shooter(self):
        return 'SHOOTING_ARMY' in self.attributes
    
    def strikes_twice(self):
        return 'const_two_attacks' in self.attributes and not self.is_shooter()
    
    def shoots_twice(self):
        return 'const_two_attacks' in self.attributes and self.is_shooter()
    
    def is_big(self):
        return 'DOUBLE_WIDE' in self.attributes
    
    def melee_penalty(self):
        return self.is_shooter() and not 'const_no_melee_penalty' in self.attributes
    
    def no_retaliation(self):
        return 'const_free_attack' in self.attributes
    
    def range_penalty(self):
        return self.is_shooter() and self.name != 'Strzelec'

In [8]:
def make_unit(name):
    return unit_type(*data.values[data.Singular.values == name][0])

In [9]:
def fight(stackA, stackB, num_iter):    
    wins = {stackA.name : [0,0],
            stackB.name : [0,0]}
    
    def units_order(s1, s2):
        temp = sorted([s1,s2], key=lambda x: x.speed, reverse=True)
        if s1.speed == s2.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp    
    
    def melee_hit(current, other):
        current.attack_melee(other, melee_penalty=current.melee_penalty())
        
        if other.is_alive() and not current.no_retaliation():
            other.attack_melee(current, melee_penalty=other.melee_penalty())
            
        if current.is_alive() and current.strikes_twice():
            current.attack_melee(other)
        return other, current
    
    def range_hit(current, other, apply_penalty):
        penalty = current.range_penalty() if apply_penalty else False
        current.attack_range(other, range_penalty=penalty)
        if current.shoots_twice() and current.shots > 0:
            current.attack_range(other, range_penalty=penalty)
        return other, current
    
    def fight_to_death(current, other):
        while current.is_alive() and other.is_alive():
            current, other = melee_hit(current, other)
        return current, other
    
    def walker_vs_shooter(walker, shooter):            
        to_walk = starting_dist - 1
        first_move = to_walk % walker.speed
        if first_move == 0:
            first_move = walker.speed
        avoid_by_move = to_walk - first_move > 10
        avoid_by_wait = False
            
        if walker.speed < shooter.speed:
            num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
        elif walker.speed > shooter.speed:
            num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
            avoid_by_wait = True
        else:
            if np.random.rand() < .5:
                num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
            else:
                num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
                    
        num_full_shots = max(0, num_shots - avoid_by_move - avoid_by_wait)
        num_half_shots = num_shots - num_full_shots
            
        for j in xrange(num_half_shots):
            range_hit(shooter, walker, apply_penalty=True)
        for j in xrange(num_full_shots):
            range_hit(shooter, walker, apply_penalty=False)
            
        current, other = walker, shooter
        return fight_to_death(current, other)
    
    starting_dist = 14
    if stackA.is_big():
        starting_dist -= 1
    if stackB.is_big():
        starting_dist -= 1
        

    for it in xrange(num_iter):
        current, other = units_order(copy(stackA), copy(stackB))
        
        if not current.is_shooter() and not other.is_shooter():
            current, other = fight_to_death(current, other)
                
        elif current.is_shooter() and other.is_shooter():
            while current.is_alive() and current.shots > 0:
                current, other = range_hit(current, other, apply_penalty=True)
            if current.is_alive():
                if other.shots == 0:
                    current, other = units_order(current, other)
                    current, other = fight_to_death(current, other)
                else:
                    current, other = walker_vs_shooter(walker=current, shooter=other)
                
        else:           
            shooter = current if current.is_shooter() else other
            walker = current if not current.is_shooter() else other
            current, other = walker_vs_shooter(walker=walker, shooter=shooter)
                
            
        winner = current if current.is_alive() else other
        wins[winner.name][0] += 1
        wins[winner.name][1] += winner.count
        
    for s in [stackA.name, stackB.name]:
        if wins[s][0]:
            wins[s][1] /= float(wins[s][0])
    
    return wins

In [10]:
# to jeszcze nie działa tak jak trzeba, są problemy z synchronizacją i seedami
def fight_multiproc(stackA, stackB, num_iter, num_proc):
    
    def units_order(s1, s2):
        temp = sorted([s1,s2], key=lambda x: x.speed, reverse=True)
        if s1.speed == s2.speed and np.random.rand() < .5:
            return reversed(temp)
        return temp    
    
    def melee_hit(current, other):
        current.attack_melee(other, melee_penalty=current.melee_penalty())
        
        if other.is_alive() and not current.no_retaliation():
            other.attack_melee(current, melee_penalty=other.melee_penalty())
            
        if current.is_alive() and current.strikes_twice():
            current.attack_melee(other)
        return other, current
    
    def range_hit(current, other, apply_penalty):
        penalty = current.range_penalty() if apply_penalty else False
        current.attack_range(other, range_penalty=penalty)
        if current.shoots_twice() and current.shots > 0:
            current.attack_range(other, range_penalty=penalty)
        return other, current
    
    def fight_to_death(current, other):
        while current.is_alive() and other.is_alive():
            current, other = melee_hit(current, other)
        return current, other
    
    def walker_vs_shooter(walker, shooter):            
        to_walk = starting_dist - 1
        first_move = to_walk % walker.speed
        if first_move == 0:
            first_move = walker.speed
        avoid_by_move = to_walk - first_move > 10
        avoid_by_wait = False
            
        if walker.speed < shooter.speed:
            num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
        elif walker.speed > shooter.speed:
            num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
            avoid_by_wait = True
        else:
            if np.random.rand() < .5:
                num_shots = to_walk / walker.speed + (to_walk % walker.speed > 0)
            else:
                num_shots = to_walk / walker.speed - (to_walk % walker.speed == 0)
                    
        num_full_shots = max(0, num_shots - avoid_by_move - avoid_by_wait)
        num_half_shots = num_shots - num_full_shots
            
        for j in xrange(num_half_shots):
            range_hit(shooter, walker, apply_penalty=True)
        for j in xrange(num_full_shots):
            range_hit(shooter, walker, apply_penalty=False)
            
        current, other = walker, shooter
        return fight_to_death(current, other)
    
    starting_dist = 14
    if stackA.is_big():
        starting_dist -= 1
    if stackB.is_big():
        starting_dist -= 1
        
    procs = []
    wins = [None]*num_proc
    for i in xrange(num_proc):
        stackA_wins = Value('i', 0)
        stackB_wins = Value('i', 0)
        wins[i] = [stackA_wins, stackB_wins]
        
    def proc_fun(wins_A, wins_B):
        wins = {stackA.name : np.array([0,0]),
                stackB.name : np.array([0,0])}
        
        for it in xrange(num_iter / num_proc):
            current, other = units_order(copy(stackA), copy(stackB))

            if not current.is_shooter() and not other.is_shooter():
                current, other = fight_to_death(current, other)

            elif current.is_shooter() and other.is_shooter():
                while current.is_alive() and current.shots > 0:
                    current, other = range_hit(current, other, apply_penalty=True)
                if current.is_alive():
                    if other.shots == 0:
                        current, other = units_order(current, other)
                        current, other = fight_to_death(current, other)
                    else:
                        current, other = walker_vs_shooter(walker=current, shooter=other)
                        
            else:           
                shooter = current if current.is_shooter() else other
                walker = current if not current.is_shooter() else other
                current, other = walker_vs_shooter(walker=walker, shooter=shooter)

            winner = current if current.is_alive() else other
            if winner.name == stackA.name:
                wins_A.value += 1
            else:
                wins_B.value += 1
    
        
    for i in xrange(num_proc):
        p = Process(target=proc_fun, args=wins[i])
        procs.append(p)
        p.start()
    
    for i in xrange(num_proc):        
        p.join()
    
    wins = {stackA.name : sum(x[0].value for x in wins),
            stackB.name : sum(x[1].value for x in wins)}
    
    return wins

In [13]:
t0 = time.time()
A = stack(make_unit(u'Pikinier'), 7015)
B = stack(make_unit(u'Tytan'), 80)

result = fight(A, B, 1000)
print result
time.time()-t0

{u'Tytan': [507, 17.01972386587771], u'Pikinier': [493, 923.9107505070994]}


0.4569880962371826

In [22]:
t0 = time.time()
A = stack(make_unit(u'Meduza'), 577)
B = stack(make_unit(u'Meduza królewska'), 392)

result = fight(A, B, 1000)
print result
time.time()-t0

{u'Meduza kr\xf3lewska': [519, 52.02504816955684], u'Meduza': [481, 60.553014553014556]}


0.4827549457550049

In [31]:
t0 = time.time()
A = stack(make_unit(u'Tytan'), 8776)
B = stack(make_unit(u'Archanioł'), 7630)

result = fight(A, B, 1000)
print result
time.time()-t0

{u'Tytan': [534, 919.8089887640449], u'Archanio\u0142': [466, 762.6008583690987]}


0.49683094024658203

In [33]:
A = stack(make_unit(u'Pikinier'), 5180000000)
B = stack(make_unit(u'Archanioł'), 80000000)

result = fight(A, B, 1000)
result

{u'Archanio\u0142': [422, 14351639.417061612],
 u'Pikinier': [578, 989555241.115917]}